# Negative Potential Vorticity Interactions with the Jet Stream Algorithm
#### Author: Alexander Lojko


## Section 1: Import Data and Prepare Filtering Domain

In [1]:
### Required Packages: Numpy, xarray, itertools, scipy, metpy, geopy and skimage

import numpy as np
import xarray as xr
import itertools 

import metpy as metpy
import metpy.xarray as mx
import metpy.calc as mc

import skimage
from skimage import feature
from skimage import measure

from scipy import ndimage
from scipy.ndimage import label, generate_binary_structure

import geopy.distance
from geopy.distance import geodesic

from collections import Counter

In [86]:
### Step 1: Import PV Data, Concat. data: 

# Open PV data (code assumes using multiple years of data saved seperately)
data_pv = xr.open_mfdataset('...',combine = 'by_coords', chunks=None)

### Step 2: Rename data variables to be more intuitive: 

data_pv = data_pv.rename({'PV_GDS0_ISBL':'pv','initial_time0_hours':'time','g0_lat_1': 'latitude','g0_lon_2':'longitude'})

data_pv = data_pv.sel(latitude=slice(80., 10.),longitude=slice(-120, -30))    # For West-Atlantic Analysis

### Step 3: Select PV variable and convert to PVU

data_pv = data_pv.pv*10**6

OSError: no files to open

In [2]:
### Step 1: Import PV Data, Concat. data: 

# pv data:
data_pv = xr.open_mfdataset('/scratch/aepayne_root/aepayne0/alojko/Proj2_Extra_PV_05_int/era5_*.nc4',combine = 'by_coords', chunks=None)

### Step 2: Rename data variables to be more intuitive: 

data_pv = data_pv.rename({'PV_GDS0_ISBL':'pv','initial_time0_hours':'time','g0_lat_1': 'latitude','g0_lon_2':'longitude'})

data_pv = data_pv.sel(latitude=slice(80., 10.),longitude=slice(-120, -30), time=slice('2017-06-05', '2017-06-20'))    # For West-Atlantic Analysis

### Step 3: Select PV variable and convert to PVU

data_pv = data_pv.pv*10**6

In [3]:
### Step 4: Define Constants for calculations below:

# Define Domain to search for Negative PV-Jet Interactions: I use [65, 25N - 100, 50W]
lat_max = 65.
lat_min = 25.
lon_min = -100
lon_max = -50

## Section 2: Identification and Length-scale Calculations of Negative PV

In [4]:
# Step 5: Filter PVU for > -0.01 PVU: Seek negative PV
data_pv_binary = data_pv.where(data_pv >= -0.01, 1).where(data_pv < -0.01, 0)

### Defining 3d array for labelling: 

# Step 6: Use ndimage to identification of negative PV objects. 
rgiObj_Struct2D = np.zeros((3,3,3)); rgiObj_Struct2D[1,:,:]=1
data_pv_labeled, num_features = label(data_pv_binary, structure=rgiObj_Struct2D)

In [5]:
### Also calculate the area size of each negative PV label. OPTIONAL STEP:

## Creation of area size mesh-grid: 

lons_mesh, lats_mesh = np.meshgrid(data_pv_binary.longitude, data_pv_binary.latitude)

EarthCircum = 40075 #[km]
dLat = np.copy(lons_mesh)
dLat[:] = EarthCircum/(360/(lats_mesh[1,0]-lats_mesh[0,0]))
dLon = np.copy(lons_mesh)

for la in range(lats_mesh.shape[0]):
    dLon[la,:] = EarthCircum/(360/(lats_mesh[1,0]-lats_mesh[0,0]))*np.cos(np.deg2rad(lats_mesh[la,0]))

Gridspacing = np.mean(np.append(dLat[:,:,None],dLon[:,:,None], axis=2))
Area = dLat*dLon
Area[Area < 0] = 0

### We have now calculated area in meters for each grid-cell (latitude weighted)

# Only calculate the area size of negative PV feature: (Can be used as alternative to major-axis length criteria)

Area_time = np.repeat(Area[np.newaxis, :, :], np.size(data_pv_binary.time), axis=0)
Area_time_pv = np.where(data_pv_binary.values==0, data_pv_binary.values, Area_time)

data_pv_labeled_areas = np.array(ndimage.sum(Area_time_pv, data_pv_labeled, np.arange(data_pv_labeled.max()+1)))

data_pv_labeled_areas = data_pv_labeled_areas[1:] # We calculate area coverage of each label here.

In [6]:
# Step 7: Calculate the length-scales of each negative PV feature. 
# Initialize dictionaries to store start and end points and label lengths
data_pv_labeled_lengths = []

# Iterate through each label and find start and end points
for label_id in range(1, num_features + 1):
    label_mask = (data_pv_labeled == label_id)
    
    # Get the indices (time, latitude, longitude) of the label
    indices = np.argwhere(label_mask)
    
    # Extract latitude and longitude values
    latitudes = indices[:, 1]
    longitudes = indices[:, 2]
    
    # Find the start and end points by finding the minimum and maximum latitude and longitude
    start_lat = latitudes.min()
    end_lat = latitudes.max()
    start_lon = longitudes.min()
    end_lon = longitudes.max()
    
    # Convert latitude and longitude values to coordinates
    start_coords = (data_pv.coords['latitude'][start_lat].values, data_pv.coords['longitude'][start_lon].values)
    end_coords = (data_pv.coords['latitude'][end_lat].values, data_pv.coords['longitude'][end_lon].values)
    
    # Calculate the great circle distance of negative PV length (i.e., account for earth curvature)
    length_km = geodesic(start_coords, end_coords).kilometers
    data_pv_labeled_lengths.append(length_km)

## Section 3: First Part of Filtering Negative PV: Data Processing Stage

In [7]:
# Convert labeled_array into xarray to keep track of coordinates if needed:
data_pv_labeled_xr = xr.zeros_like(data_pv) + data_pv_labeled

# Use defined domain mask to identify negative PV features overlapping with domain.

domain_mask = (data_pv_labeled_xr.coords['latitude'] >= lat_min) & (data_pv_labeled_xr.coords['latitude'] <= lat_max) & \
              (data_pv_labeled_xr.coords['longitude'] >= lon_min) & (data_pv_labeled_xr.coords['longitude'] <= lon_max)

In [8]:
### New section, create a list of latlon which is indexed by label number: 

data_pv_labeled_xr_nans = data_pv_labeled_xr.where(data_pv_labeled_xr!=0)
data_pv_labeled_xr_stacked = data_pv_labeled_xr_nans.stack(coordinates=['time', 'latitude','longitude']) 

### Remove the NaNs (or zeros):

data_pv_labeled_xr_stacked = data_pv_labeled_xr_stacked[data_pv_labeled_xr_stacked.notnull()]

In [9]:
### Remove repeating instances of negative PV labels: 

index_no, time_index = np.unique(data_pv_labeled_xr_stacked.values, return_index=True)
data_pv_label_time = data_pv_labeled_xr_stacked.isel(coordinates=time_index).time.to_numpy() # each labels time

# use Counter to count occurrences of each date
date_counts = Counter(data_pv_label_time)

# create new array with counts of each unique date: How many NPV labels occur for a particular time-step? 
unique_dates = list(set(data_pv_label_time))
unique_dates = np.sort(unique_dates)
data_pv_count_array = [date_counts[date] for date in unique_dates]

In [10]:
# Repeat the values in the initial Xarray based on the repeat_counts array:

repeated_pv_data = np.repeat(data_pv.values, data_pv_count_array, axis=0)

# Create a new 3D Xarray with repeated indexes along the 'x' dimension
repeated_pv_data = xr.DataArray(repeated_pv_data, dims=('time', 'latitude', 'longitude'))

repeated_pv_label = np.repeat(data_pv_labeled_xr_nans.values, data_pv_count_array, axis=0)
repeated_pv_label = xr.DataArray(repeated_pv_label, dims=('time', 'latitude', 'longitude')) 

repeated_pv_label_number = repeated_pv_label
repeated_pv_label_number['time'] = repeated_pv_label_number['time'] + 1 # set times to match labels

In [11]:
### we have repeated label array, now doing the same thing for domain mask array:

# Get the number of time steps (size of the time dimension) in the 3D xarray
num_time_steps = repeated_pv_label.sizes['time']

# Repeat the 2D xarray along the new 'time' dimension
repeated_mask = xr.concat([domain_mask] * num_time_steps, dim='time')

# Assign coordinates from the 3D xarray to the new 'time' dimension
repeated_mask['time'] = repeated_pv_label.coords['time']

## Section 4: Filter out Negative PV features not inside domain of interest

In [40]:
## Next step:
# for repeated_pv_label, where nan, set to 0, where number set to 1
# for repeated_mask, where False, set to 0, where True, set to 1
# Then, add the 2 matrix up, for each label (time) where a value of 2 appears, we keep. 

repeated_pv_label = repeated_pv_label.fillna(0)
repeated_pv_label = repeated_pv_label.where(repeated_pv_label <= 1, 1)

repeated_mask = repeated_mask.astype(int)
repeated_mask['time'] = repeated_mask['time']

In [41]:
### Now let's add up the 2 matrix and search for values of 2 per label

label_mask_filter = repeated_pv_label + repeated_mask

In [42]:
### We now have values that add up: 
label_mask_filter_max = []

for i in range(label_mask_filter.shape[0]):
    label_mask_filter_max.append(np.max(label_mask_filter[i,:,:].values))
    
repeated_pv_label_number = repeated_pv_label_number.fillna(0)
time_labels = np.arange(1,np.size(repeated_pv_label_number.time)+1)

In [43]:
# This section searches for negative PV features inside the domain by seeking values of 2: Where there is 2 means NPV label overlaps with domain.
individual_pv_labels = xr.zeros_like(repeated_pv_label_number)

# Iterate through each time step
for time_idx in range(len(repeated_pv_label_number.time)):
    label_filter = time_labels[time_idx]
    
    # Create a boolean mask to select the label for the current time step
    label_mask = repeated_pv_label_number.sel(time=repeated_pv_label_number.time[time_idx]) == label_filter
    
    # Set the selected label in the filtered data, and all other labels to zero
    individual_pv_labels[time_idx] = label_mask.astype(int)
    

In [44]:
### Quick check to make sure still using lat/lon coords rather than numpy (each label has its own time-dimension)

individual_pv_labels['latitude'] = data_pv.coords['latitude']
individual_pv_labels['longitude'] = data_pv.coords['longitude']

In [17]:
### Check where NPV label intersects with domain:

pv_label_mask_filter = individual_pv_labels + repeated_mask

# Create a new 1D vector with the same size as the time dimension, initialized with zeros
new_vector = xr.DataArray(np.zeros(pv_label_mask_filter.time.size), dims='time', coords={'time': pv_label_mask_filter.time})

# Check if 2 occurs in the 3D xarray along the time dimension
presence_of_2 = (pv_label_mask_filter == 2).any(dim=('latitude', 'longitude'))

# Assign 1 to the new vector for time steps where 2 occurs
new_vector[presence_of_2] = 1

### We now have the index where labels overlap with the mask domain

In [47]:
selected_label = individual_pv_labels.sel(time=new_vector == 1) # labeled arrays
selected_pv = repeated_pv_data.sel(time=new_vector == 1) # regular pv field
selected_time_index_labels = data_pv_label_time[new_vector==1] # times

In [48]:
label_lengths_km_filtered = np.array(data_pv_labeled_lengths)[new_vector.values.astype(int) == 1]

## Section 5: Filter out small negative PV features, only keep synoptic scale features

In [49]:
# Find 98th percentile, where the 98th percentile is treated as synoptic-scale. 
# In Lojko et al., 2024, climatological length scale in ERA5 is about 1650 km. For now, let's just keep upper 98th percentile.

percentile_threshold = np.percentile(label_lengths_km_filtered, 98)

# Let's filter at percentile threshold of length scale.  

percentile_label_length_index = np.where(label_lengths_km_filtered > percentile_threshold)

label_lengths_km_filtered_percentile = label_lengths_km_filtered[percentile_label_length_index]
selected_data_percentile = selected_pv.where(np.isin(selected_pv,  percentile_label_length_index), 0)

In [50]:
### Data preperation: 

selected_label_filt = selected_label.isel(time=np.vstack(percentile_label_length_index).squeeze())
selected_pv_filt = selected_pv.isel(time=np.vstack(percentile_label_length_index).squeeze())

### Make xr like to use isel.

selected_time_index_labels_filt = selected_time_index_labels[np.vstack(percentile_label_length_index).squeeze()]

In [51]:
### Reconstruct the selected_pv_filt dataset as before, now make time axis instead of axis being index of label. 
selected_pv_filt['latitude'] = selected_label_filt.coords['latitude']
selected_pv_filt['longitude'] = selected_label_filt.coords['longitude']
selected_pv_filt['time'] = selected_time_index_labels_filt

selected_label_filt['time'] = selected_time_index_labels_filt # recently added in, removes index of labels.

In [52]:
### Relabel remaining labels from 1 - N:
unique_selected_label_filt = selected_label_filt.groupby('time').first()

# Restructuring of time data:
labeled_array_new, num_features_new = label(selected_label_filt, structure=rgiObj_Struct2D)
labeled_array_new_xr = xr.zeros_like(selected_label_filt) + labeled_array_new 

In [53]:
### First some re-stacking of labeled data filtered:
### New section, create a list of latlon which is indexed by label number: 
xr_remove_small_label_filt_nans = labeled_array_new_xr.where(labeled_array_new_xr!=0)
xr_remove_small_label_stacked = xr_remove_small_label_filt_nans.stack(coordinates=['time', 'latitude','longitude']) 

### Remove the NaNs (or zeros):
xr_remove_small_label_stacked = xr_remove_small_label_stacked[xr_remove_small_label_stacked.notnull()]

In [54]:
### Last step, get a list of coordinates of all Neg PV labels remaining:

lats_list = [] 
lons_list = []
times_list = [] 

for i in range(int(np.max(xr_remove_small_label_stacked.to_numpy()+1))):
    lats_list.append(xr_remove_small_label_stacked['latitude'].where(xr_remove_small_label_stacked == i, drop=True).to_numpy())
    lons_list.append(xr_remove_small_label_stacked['longitude'].where(xr_remove_small_label_stacked == i, drop=True).to_numpy())

### Combine lats and lons: 

listing_label_1PVU = []   # recycling the same name as before. But this time with corrected coordinates: 
for a, b in zip(lats_list, lons_list):
    listing_label_1PVU.append(list(zip(a,b)))

listing_label_1PVU = listing_label_1PVU[1:] # first index is no value.


### Section 6: Preperation of 2 PVU data

In [55]:
### 2PVU time:

data_pv_smooth = selected_pv_filt # Filter times to match -1 PVU days of interest. 
data_pv_smooth = mc.smooth_gaussian(data_pv_smooth, 15)  # Smooth to reduce instances of very small 2 pvu filaments

In [57]:
### Use scikit image package to ID lines of 2 PVU: (MUST BE A LIST!)

listing_2PVU = []
listing_2PVU_filt = [] # This list will filter out closed contours.

for i in range(np.size(data_pv_smooth.time)): # time size should be the same for pv_pos_2
    listing_2PVU.append(measure.find_contours(data_pv_smooth.values[i], 2))
    listing_2PVU_filt.append([c for c in listing_2PVU[i] if c[0,0] != c[-1,0] and c[0,1] != c[-1,1]])
    
### Concatenate the middle nested list axis (contour number) due to scikit image seperating individual contours: 

listing_2PVU_filt = [list(itertools.chain(*sub)) for sub in listing_2PVU_filt]

### At this point, we have the coordinates, but they will need to be transformed back onto the lat-lon grid. This will be 
### done shortly. However, there is a small subset of dates that have no unclosed contours identified. When calculating
### the distance between -1 PVU coords and 2 PVU coords, there needs to be at least 1 coordinate for calculation to work.
### Hence, first, let us first filter out any time-steps in the 2 PVU list with no values. 

In [58]:
### Creating script prior to Jet - to - negative PV feature detection, eliminate all instances of 0 data size in list. 

size_append_2PVU = [] 
for i in range(np.size(listing_2PVU_filt)):
    size_append_2PVU.append(np.size(listing_2PVU_filt[i])) # Only 2 PVU list has instances of 0 data. 

zero_index_2PVU = [] # indicies where value == 0
for i, x in enumerate(size_append_2PVU):
    if x == 0:
        zero_index_2PVU.append(i)
        
zero_index_2PVU_label = list(np.asarray(zero_index_2PVU) + 1)
        
non_zero_index_2PVU = [] # indicies where value != 0
for i, x in enumerate(size_append_2PVU):
    if x != 0:
        non_zero_index_2PVU.append(i)
        
non_zero_index_2PVU_label = list(np.asarray(non_zero_index_2PVU) + 1)

### Next step, filter out the ID'd zero_index_2PVU in  the 2 PVU list, -1 PVU list and in time axis. 

listing_neg_1_pv_filt = [x for i, x in enumerate(listing_label_1PVU) if i not in zero_index_2PVU]
listing_pos_2_pv_filt = [x for i, x in enumerate(listing_2PVU_filt) if i not in zero_index_2PVU]

#np.shape(listing_test_2_filt)

size_append_2PVUnew = [] 
for i in range(np.size(listing_neg_1_pv_filt)):
    size_append_2PVUnew.append(np.size(listing_pos_2_pv_filt[i])) # Only 2 PVU list has instances of 0 data. 

/home/alojko/miniconda3/envs/cartopy-pangeo/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3202: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).size


In [59]:
### Keep track of seperate time array, make sure has same index as the -1 PVU and 2 PVU list we are making:
repeated_time_nonzero = selected_time_index_labels_filt[non_zero_index_2PVU]

# In order to change lat/lon, need to vstack to reduce instances of 'array' in nested list: (repeat process)  

listing_2PVU_clean =[]
for i in range(np.size(listing_pos_2_pv_filt)): 
    listing_2PVU_clean.append(np.vstack(listing_pos_2_pv_filt[i])) 

In [60]:
# Coordinate transformation from numpy index to lat/lon: 

lons_2PVU = []
lats_2PVU = [] 
coord = [] 

for contour in listing_2PVU_clean:
    lons_2PVU.append(data_pv.longitude.values[0]+contour[:, 1]*0.5)  # 0.5 because of reslution of 2PVU line used. 
    lats_2PVU.append(data_pv.latitude.values[0]+contour[:, 0]*-0.5)

# FOR LON: -(MOST WEST LONGITUDE)+CONTOUR*(GRID RESOLUTION)      --- assuming longitude is negative. 
# FOR LAT: +(MOST NORTH LATITUDE)+CONTOUR*(-GRID RESOLUTION)    --- assuming positive latitude, note negative sign in grid res. 

listing_2PVU_final = []   # recycling the same name as before. But this time with corrected coordinates: 
for a, b in zip(lats_2PVU, lons_2PVU):
    listing_2PVU_final.append(list(zip(a,b)))

### Section 7: Find minimum distance of each negative PV label to a continious 2 PVU contour:

In [61]:
import timeit
start = timeit.default_timer()

dist = [] 
coord_append = [] 
min_dist = [] 
min_index = [] 
coord_append_min = [] 

for i in range(np.size(listing_2PVU_final)): # Time Axis 
    for value1 in listing_2PVU_final[i]: # For each coordinate pair in PV2
            for value2 in listing_neg_1_pv_filt[i]:  # For each coordinate pair in negPV1
                dist.append(geopy.distance.great_circle(value1, value2).km) # Calculate distance 
                coord_append.append([value1, value2]) # all coordinates appended for each time-step. 
    min_dist.append(np.min(dist)) # Find minimum distance at each time. 
    min_index.append(np.argmin(dist))
    coord_append_min.append(coord_append[min_index[i]]) # set to i to find minimum mid-point at each time-step. 
    dist = []  # Clear distance array to save memory and for minimum distance func. to work. 
    coord_append = [] 

stop = timeit.default_timer()

print('Time: ', stop - start)  

Time:  29.045618513599038


In [62]:
### Make into numpy array:

min_dist_array = np.array(min_dist)
min_index_array = np.array(min_index)

In [63]:
### Filtering values not within 100 km:

min_dist_keep = np.where(min_dist_array <= 100) 
min_dist_keep = min_dist_keep[0]

min_dist_discard = np.where(min_dist_array >= 100) 
min_dist_discard = min_dist_discard[0]

In [64]:
### Also get mid-points:
midpoint_coords = []
just_2PVU = []
just_1PVU = []
for i in range(np.size(listing_2PVU_final)): # Time Axis
    just_2PVU.append(np.array(coord_append_min[i][0]))
    just_1PVU.append(np.array(coord_append_min[i][1]))
    midpoint_coords.append((np.array(coord_append_min[i][0]) + np.array(coord_append_min[i][1]) )/2)
    
midpoint_coords = np.vstack(midpoint_coords)

midpoint_coords_100 = midpoint_coords[min_dist_keep]
min_dist_100 = np.array(min_dist)[min_dist_keep]
repeated_time_nonzero_100 = repeated_time_nonzero[min_dist_keep]

## Final filtering, if multiple NPV features near jet stream for single time-step, only keep one closest to the jet stream.

In [67]:
### Now check 
# Define your conditions
lat_condition = (midpoint_coords_100[:, 0] >= lat_min) & (midpoint_coords_100[:, 0] <= lat_max)
lon_condition = (midpoint_coords_100[:, 1] >= lon_min) & (midpoint_coords_100[:, 1] <= lon_max)

# Apply the conditions
filtered_midpoint_coords_100 = midpoint_coords_100[lat_condition & lon_condition]

# Also get index of where condition is satisfied:
condition_satisfied = lat_condition & lon_condition

# Get indices of coordinates that satisfy the conditions
indices = np.where(condition_satisfied)[0]

# Apply indice to the -1PVU and 2PVU coords:
just_1PVU_filt = np.vstack(just_1PVU)[indices]
just_2PVU_filt = np.vstack(just_2PVU)[indices]

# Also, to the remaining times:
repeated_time_nonzero_filt = repeated_time_nonzero_100[indices]
min_dist_100_filt = min_dist_100[indices]
midpoint_coords_100_filt = midpoint_coords_100[indices]

In [68]:
### Filter out repeating dates:

# Combine dates and numbers using zip
combined_data = list(zip(repeated_time_nonzero_filt, min_dist_100_filt))

# Create a dictionary to store the minimum number for each unique date
min_numbers = {}
for date, distance in combined_data:
    if date not in min_numbers or distance < min_numbers[date]:
        min_numbers[date] = distance

# Create filtered arrays
filtered_dates, filtered_numbers = zip(*min_numbers.items())

# Convert the filtered data to numpy arrays
filtered_dates = np.array(filtered_dates)
filtered_numbers = np.array(filtered_numbers)